# 0. Download Dataset

In [8]:
!python -m wget https://motchallenge.net/data/MOT17.zip


Saved under MOT17.zip


In [1]:
import os

path = os.getcwd()
print(path)

C:\Users\Admin\Track by Detect


In [2]:
import zipfile

with zipfile.ZipFile('MOT17.zip', 'r') as zip_ref:
    zip_ref.extractall(path)

NameError: name 'path' is not defined

# 1. Import Libraries

In [19]:
!pip install ultralytics -q

In [2]:
!pip install pandas

In [3]:
import os
import yaml
import shutil
import configparser
import pandas as pd
import ultralytics
ultralytics.checks()

from tqdm import tqdm
from ultralytics import YOLO

Ultralytics YOLOv8.1.3 🚀 Python-3.11.7 torch-2.1.2+cpu CPU (12th Gen Intel Core(TM) i5-12400F)
Setup complete ✅ (12 CPUs, 15.8 GB RAM, 175.7/237.8 GB disk)


# 2. Convert to YOLO format

In [45]:
def convert_to_yolo_format(bb, img_width, img_height):
    x_center = bb['bb_left'] + (bb['bb_width'] / 2)
    y_center = bb['bb_top'] = (bb['bb_height'] / 2)

    # Normalize the coordinates by the dimensions of the image
    x_center /= img_width
    y_center /= img_height
    bb_width_nomalized = bb['bb_width'] / img_width
    bb_height_normalized = bb['bb_height'] / img_height

    # Clip the values to make sure they are between 0 and 1
    x_center = max(min(x_center, 1), 0)
    y_center = max(min(y_center, 1), 0)
    bb_width_nomalized = max(min(bb_width_nomalized, 1), 0)
    bb_height_normalized = max(min(bb_height_normalized, 1), 0)

    return (x_center, y_center, bb_width_nomalized, bb_height_normalized)

In [46]:
def process_folder(folder_path):
    # Read image dimensions from seqinfo.ini
    config = configparser.ConfigParser()
    config.read(os.path.join(folder_path, 'seqinfo.ini'))
    img_width = int(config['Sequence']['imWidth'])
    img_height = int(config['Sequence']['imHeight'])

    # Load ground truth data
    gt_path = os.path.join(folder_path, 'det/det.txt')
    gt_data = pd.read_csv(
        gt_path,
        header=None,
        names=['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'conf', 'class', 'visibility']
    )

    labels_folder = os.path.join(folder_path, 'labels')
    os.makedirs(labels_folder, exist_ok=True)

    for frame_number in gt_data['frame'].unique():
        frame_data = gt_data[gt_data['frame'] == frame_number]
        label_file = os.path.join(labels_folder, f'{frame_number:06d}.txt')

        with open(label_file, 'w') as file:
            for _, row in frame_data.iterrows():
                yolo_bb = convert_to_yolo_format(row, img_width, img_height)
                file.write(f'0 {yolo_bb[0]} {yolo_bb[1]} {yolo_bb[2]} {yolo_bb[3]}\n')

In [51]:
def process_all_folders(base_directory):
    # List all subdirectories in the base directory
    for folder_name in tqdm(os.listdir(base_directory)):
        folder_path = os.path.join(base_directory, folder_name)

        # Delete folder not contain 'FRCNN' in name
        if 'FRCNN' not in folder_name:
            shutil.rmtree(folder_path)
            continue

        if os.path.isdir(folder_path):
            process_folder(folder_path)

In [52]:
process_all_folders(os.path.join('MOT17', 'train'))
process_all_folders(os.path.join('MOT17', 'test'))

100%|██████████| 21/21 [00:10<00:00,  2.04it/s]


# 3. Move file

In [53]:
def rename_and_move_files(src_folder, dst_folder, folder_name, file_extension):
    for filename in os.listdir(src_folder):
        if filename.endswith(file_extension):
            # Include folder name in the new filename
            new_filename = f'{folder_name}_{filename}'
            shutil.move(os.path.join(src_folder, filename), os.path.join(dst_folder, new_filename))

In [54]:
def move_files_all_folders(base_directory):
    images_dir = os.path.join(base_directory, 'images')
    labels_dir = os.path.join(base_directory, 'labels')
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    for folder_name in tqdm(os.listdir(base_directory)):
        if folder_name in ['images', 'labels']:   # Skip these folders
            continue

        folder_path = os.path.join(base_directory, folder_name)
        if os.path.isdir(folder_path):
            rename_and_move_files(os.path.join(folder_path, 'img1'), images_dir, folder_name, '.jpg')
            rename_and_move_files(os.path.join(folder_path, 'labels'), labels_dir, folder_name, '.txt')

In [55]:
move_files_all_folders(os.path.join('MOT17', 'train'))
move_files_all_folders(os.path.join('MOT17', 'test'))

100%|██████████| 9/9 [00:02<00:00,  3.56it/s]


In [56]:
def delete_subfolders(base_directory):
    for folder_name in os.listdir(base_directory):
        folder_path = os.path.join(base_directory, folder_name)
        if os.path.isdir(base_directory) and folder_name not in ['images', 'labels']:
            shutil.rmtree(folder_path)
            print(f'Remove folder: {folder_name}')

In [57]:
delete_subfolders(os.path.join('MOT17', 'train'))
delete_subfolders(os.path.join('MOT17', 'test'))

Remove folder: MOT17-02-FRCNN
Remove folder: MOT17-04-FRCNN
Remove folder: MOT17-05-FRCNN
Remove folder: MOT17-09-FRCNN
Remove folder: MOT17-10-FRCNN
Remove folder: MOT17-11-FRCNN
Remove folder: MOT17-13-FRCNN
Remove folder: MOT17-01-FRCNN
Remove folder: MOT17-03-FRCNN
Remove folder: MOT17-06-FRCNN
Remove folder: MOT17-07-FRCNN
Remove folder: MOT17-08-FRCNN
Remove folder: MOT17-12-FRCNN
Remove folder: MOT17-14-FRCNN


# 4. Create yaml file

In [4]:
class_labels = [
    'objects'
]
dataset_root_dir = os.path.join(
    os.getcwd(),
    'MOT17'
)
yolo_yaml_path = os.path.join(
    dataset_root_dir,
    'mot17_data.yml'
)

data_yaml = {
    'path': dataset_root_dir,
    'train': 'train/images',
    'val': 'test/images',
    'nc': len(class_labels),
    'names': class_labels
}

with open(yolo_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

# 5. Training

In [5]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8s.pt')

# Config
epochs = 10
batch_size = -1   # Auto scale base on GPU memory
img_size = 640
project_name = 'models/yolo'
name = 'yolov8s_mot17_det'

# Train the model
results = model.train(
    data = yolo_yaml_path,
    epochs = epochs,
    batch = batch_size,
    imgsz = img_size,
    project = project_name,
    name = name
)

Ultralytics YOLOv8.1.3 🚀 Python-3.11.7 torch-2.1.2+cpu CPU (12th Gen Intel Core(TM) i5-12400F)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\Admin\Track by Detect\MOT17\mot17_data.yml, epochs=10, time=None, patience=50, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=models/yolo, name=yolov8s_mot17_det, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_

train: Scanning C:\Users\Admin\Track by Detect\MOT17\train\labels.cache... 5316 images, 0 backgrounds, 0 corrupt: 100%|
val: Scanning C:\Users\Admin\Track by Detect\MOT17\test\labels.cache... 5908 images, 11 backgrounds, 0 corrupt: 100%|██


Plotting labels to models\yolo\yolov8s_mot17_det\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to models\yolo\yolov8s_mot17_det
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      1.701      1.467      1.236         51        640: 100%|██████████| 333/333 [1:15:31<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 185/185 [19:


                   all       5919     110141      0.728       0.55      0.621      0.316

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       5919     110141      0.851      0.701      0.775      0.426

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10         0G     0.8021     0.5268     0.8771         49        640: 100%|██████████| 333/333 [1:04:53<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 185/185 [17:


                   all       5919     110141      0.858      0.698       0.78      0.441

10 epochs completed in 14.847 hours.
Optimizer stripped from models\yolo\yolov8s_mot17_det\weights\last.pt, 22.5MB
Optimizer stripped from models\yolo\yolov8s_mot17_det\weights\best.pt, 22.5MB

Validating models\yolo\yolov8s_mot17_det\weights\best.pt...
Ultralytics YOLOv8.1.3 🚀 Python-3.11.7 torch-2.1.2+cpu CPU (12th Gen Intel Core(TM) i5-12400F)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 185/185 [14:


                   all       5919     110141      0.858      0.698       0.78      0.441
Speed: 1.9ms preprocess, 133.8ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to models\yolo\yolov8s_mot17_det


# 6. Evaluation

In [6]:
from ultralytics import YOLO

# Load the trained model
model_path = os.path.join(
    project_name, name, 'weights/best.pt'
)
model = YOLO(model_path)

metrics = model.val(
    project = project_name,
    name = 'detect/val'
)

Ultralytics YOLOv8.1.3 🚀 Python-3.11.7 torch-2.1.2+cpu CPU (12th Gen Intel Core(TM) i5-12400F)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning C:\Users\Admin\Track by Detect\MOT17\test\labels.cache... 5908 images, 11 backgrounds, 0 corrupt: 100%|██
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 370/370 [15:


                   all       5919     110141      0.858      0.698       0.78      0.441
Speed: 1.4ms preprocess, 140.7ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to models\yolo\detect\val


# 7. Inference

In [7]:
path

'C:\\Users\\Admin\\Track by Detect'

In [9]:
sample_path = os.path.join(path, 'MOT17-01-FRCNN_000001.jpg')
results = model.predict(
    sample_path,
    project = project_name,
    name = 'detect/predict',
    save = True
)


image 1/1 C:\Users\Admin\Track by Detect\MOT17-01-FRCNN_000001.jpg: 384x640 10 objectss, 152.9ms
Speed: 13.1ms preprocess, 152.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to models\yolo\detect\predict


In [4]:
#shutil.rmtree(os.path.join(path, 'models'))